In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False
# 매직명령어 => 쥬피터노트북에서 그래프 삽입 기능 
%matplotlib inline
# 글꼴 선명화 
%config InlineBackend.figure_format = 'retina'

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation

# CNN : 커널을 통해 합성곱을 구하는 것. 이미지의 특징이 두드러 지게 한다.
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv1D

# MaxPooling : 커널내에서 가장 큰 값을 추출하는 방식으로 불필요한 부분을 제거한다.
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool1D

# 1차원으로 변환하는 것
from tensorflow.keras.layers import Flatten

# Dropout : 이미자나 영상, 음파 등의 데이터는 오랫동안 학습을 진행 시켜야 한다.
# 하지만 너무 빨리 과적합이 발생되면 조기 중단 때문에 학습 횟수가 줄어들 수 있다.
# 이에 은닉의 노드를 일부 사용하지 않으므로써 과적합이 빨리 오는 것을 예방하고
# 오랫동안 학습이 가능하다.
from tensorflow.keras.layers import Dropout

# Embadding : 단어의 수를 조정한다.
from tensorflow.keras.layers import Embedding

# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 저장된 학습모델을 복원한다.
from tensorflow.keras.models import load_model

# 모델을 자동 저장한다.
from tensorflow.keras.callbacks import ModelCheckpoint
# 성능이 더이상 좋아지지 않을 경우 중단 시킨다.
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# 문장을 잘라준다.
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장 데이터의 단어 데이터의 수를 동일한 수로 맞춰준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 가지고 단어 사전을 만들어준다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 문자열 => 숫자
from sklearn.preprocessing import LabelEncoder

# 전체데이터를 학습용과 검증으로 나눈다.
from sklearn.model_selection import train_test_split

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

### 문장 자르기

In [2]:
# 테스트 문자열
text = '해보지 않으면 해낼 수 없다'

In [3]:
# 주어진 문자열을 토큰화 한다.
result = text_to_word_sequence(text)
print(f'원본 : {text}')
print(f'토큰화 : {result}')

원본 : 해보지 않으면 해낼 수 없다
토큰화 : ['해보지', '않으면', '해낼', '수', '없다']


###  단어 빈도수

In [4]:
docs = [
    '먼저 텍스트와 각 단어를 나누어 토큰화 합니다',
    '텍스트의 단어로 토큰화 해야 딥러닝에서 인식합니다',
    '토큰화 한 결과는 딥러닝에서 사용할 수 있다.'
]

In [5]:
# 토큰화 함수를 통해 전처리를 한다.
token = Tokenizer()
token.fit_on_texts(docs)

In [6]:
# 단어의 빈도수
# 전체에서 어떤 단어가 몇 개씩 나왔는지...
token.word_counts

OrderedDict([('먼저', 1),
             ('텍스트와', 1),
             ('각', 1),
             ('단어를', 1),
             ('나누어', 1),
             ('토큰화', 3),
             ('합니다', 1),
             ('텍스트의', 1),
             ('단어로', 1),
             ('해야', 1),
             ('딥러닝에서', 2),
             ('인식합니다', 1),
             ('한', 1),
             ('결과는', 1),
             ('사용할', 1),
             ('수', 1),
             ('있다', 1)])

In [7]:
# 전체 문장의 개수
token.document_count

3

In [8]:
# 각 단어가 몇 개의 문자에서 나왔는지...
token.word_docs

defaultdict(int,
            {'텍스트와': 1,
             '나누어': 1,
             '토큰화': 3,
             '합니다': 1,
             '단어를': 1,
             '각': 1,
             '먼저': 1,
             '해야': 1,
             '단어로': 1,
             '딥러닝에서': 2,
             '인식합니다': 1,
             '텍스트의': 1,
             '한': 1,
             '수': 1,
             '사용할': 1,
             '있다': 1,
             '결과는': 1})

In [9]:
# 각 단어에 부여된 인덱스(단어 사전)
token.word_index

{'토큰화': 1,
 '딥러닝에서': 2,
 '먼저': 3,
 '텍스트와': 4,
 '각': 5,
 '단어를': 6,
 '나누어': 7,
 '합니다': 8,
 '텍스트의': 9,
 '단어로': 10,
 '해야': 11,
 '인식합니다': 12,
 '한': 13,
 '결과는': 14,
 '사용할': 15,
 '수': 16,
 '있다': 17}

###  딥러닝

In [10]:
# 평가글
docs = [
    '너무 재미있네요',
    '최고에요',
    '참 잘 만든 영화에요',
    '추천하고 싶은 영화입니다',
    '한번 더 보고 싶네요',
    '글쎄요',
    '별로에요',
    '생각보다 지루하네요',
    '연기가 어색해요',
    '재미없어요'
]

In [11]:
# 결과데이터
classes = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [12]:
# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)

In [13]:
# 단어 사전을 출력해본다.
token.word_index

{'너무': 1,
 '재미있네요': 2,
 '최고에요': 3,
 '참': 4,
 '잘': 5,
 '만든': 6,
 '영화에요': 7,
 '추천하고': 8,
 '싶은': 9,
 '영화입니다': 10,
 '한번': 11,
 '더': 12,
 '보고': 13,
 '싶네요': 14,
 '글쎄요': 15,
 '별로에요': 16,
 '생각보다': 17,
 '지루하네요': 18,
 '연기가': 19,
 '어색해요': 20,
 '재미없어요': 21}

In [14]:
# 각 문장을 토큰화 시킨 데이터를 단어 인덱스로 변화한다.
X = token.texts_to_sequences(docs)
X

[[1, 2],
 [3],
 [4, 5, 6, 7],
 [8, 9, 10],
 [11, 12, 13, 14],
 [15],
 [16],
 [17, 18],
 [19, 20],
 [21]]

In [15]:
# 각 리스트의 데이터의 개수를 제일 많은 문장의 개수로 맞춰준다.
cnt_max = 0
for a1 in X :
    if len(a1) > cnt_max :
        cnt_max = len(a1)
padded_x = pad_sequences(X, cnt_max)
padded_x

array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 4,  5,  6,  7],
       [ 0,  8,  9, 10],
       [11, 12, 13, 14],
       [ 0,  0,  0, 15],
       [ 0,  0,  0, 16],
       [ 0,  0, 17, 18],
       [ 0,  0, 19, 20],
       [ 0,  0,  0, 21]])

In [16]:
# 원핫 인코딩을 위한 단어의 개수를 파악한다.
# 단어사전에서 제일 작은 값이 1이므로 전체 단어수에 1을 더해준다.
word_size = len(token.word_index) + 1
word_size

22

In [17]:
# 2진 분류 옵션들
out_nodes = 1
loss_function = 'binary_crossentropy'
activation_function = 'sigmoid'

# 다중 분류 옵션들
# out_nodes = len(y.value_counts())
# loss_function = 'categorical_crossentropy'
# activation_function = 'softmax'

# 회귀
# 출력 결과를 하나만 뽑아서 Series로 나왔다면...
# 1로 설정한다.
# if type(y) is pd.core.series.Series:
#     out_nodes = 1
# else : 
#     out_nodes = y.shape[1]
    
# loss_function = 'mean_squared_error'

In [18]:
# 학습 모델을 구성한다.
model = Sequential()

# Embadding 층
# 단어 값들을 원핫 인코딩을 수행한 수 결과와 상관관계가 높은(빈도수 높은)
# 단어 컬럼을 기준으로 지정한 컬럼수 만큼 선택하여 다른 컬럼의 데이터를 압축하는 은닉층
# input_length : 입력층으로 들어오는 각 문장단 단어의 개수
# 위에서 padding 작업을 통해 각 문장의 단어를 전부 4개로 맞췄기 때문에 4로 설정했다.
# word_size : 들어온 문장데이터를 원핫 인코딩할 데이터 개수(단어의수 +1)
# input_length 만큼의 문장 하나의 데이터([1,2,3,4])가 들어오면
# 각 단어를 뜻하는 숫자를 각각 원핫 인코딩한다.
# [[0, 1, 0, 0, 0]
#  [0, 0, 1, 0, 0]
#  [0, 0, 0, 1, 0]
#  [0, 0, 0, 0, 1]]
# 그 후에 결과데이터와의 상관관계를 보고 8개만 추출하여 최종데이터를 만든다.
model.add(Embedding(word_size, 8, input_length=4))
# 다차원으로 변화된 데이터를 1차원으로 변환한다.
model.add(Flatten())

# 출력층
model.add(Dense(out_nodes))
model.add(Activation(activation_function))

In [22]:
# 모델 컴파일
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              176       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 학습
model.fit(padded_x, classes, epochs=20)

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6989 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6966 - accuracy: 0.4000
Epoch 3/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6944 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6922 - accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6899 - accuracy: 0.7000
Epoch 6/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6877 - accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6855 - accuracy: 0.7000
Epoch 8/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6833 - accuracy: 0.7000
Epoch 9/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6811 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6789 - accuracy: 0.8000
Epoch 11/20
1/1 [=======================